In [1]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

### Ingest SIA Annual Report 2023 PDF

In [4]:
docs = PyPDFLoader('./SIA_Annual_Report_2023.pdf').load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

### Generate Embeddings and store in Chroma Vector Store (Local)

In [18]:
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=OpenAIEmbeddings()
)
vectorstore.add_documents(docs[:500]);

### Create QA Bot based on the Vector Store

In [19]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0),
    vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
)

In [20]:
# Test the QA chain
result = qa.invoke(({"question": "What is SIA Mission Statement?"}))
print(result['answer'])

 Singapore Airlines is dedicated to providing high-quality air transportation services and maximizing returns for shareholders and employees.


In [21]:
from langchain.agents import (load_tools, initialize_agent)
from langchain.agents import Tool

### Create a tool based on the QA Bot and give it a description
#### *** Description is important as it is how the agent decide which tool to use based on the question

In [22]:
sia_qa_tool = Tool(
    name="SIA QA System",
    func=qa.run,
    description="Provides responses to the questions about Singapore Airlines (SIA), financial performance and report for 2022/2024 and its future plans",
    return_direct=True
)

#### Load tools serpapi (google search) and our SIA QA Tool/Bot

In [23]:
tools = load_tools(["serpapi"]) + [sia_qa_tool]

#### Here you can see serpapi(google search) tool's description

In [24]:
tools[0].description

'A search engine. Useful for when you need to answer questions about current events. Input should be a search query.'

#### Create Agent with the Tools and LLM (OpenAI GPT)

In [25]:
agent = initialize_agent(tools, OpenAI(temperature=0), agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

/Users/weishen/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


#### Test out the Agent by giving it some questions

In [26]:
sia_qa_result  = agent.invoke("How is SIA financial performance in 2022/2023?")
sia_qa_result['output'] # Should get the answer from the SIA QA Tool



> Entering new AgentExecutor chain...
 I should use the SIA QA System to get accurate information about SIA's financial performance.
Action: SIA QA System
Action Input: SIA financial performance 2022/2023
Observation:  The projected financial performance of SIA for 2022/2023 includes a record passenger carriage of more than 26 million, a capacity of 79%, and record revenues, operating profits, and net profits of $17,775 million, $2,692 million, and $2,157 million, respectively. The Board of Directors also recommends a final dividend of 28 cents per share for the financial year.


> Finished chain.


' The projected financial performance of SIA for 2022/2023 includes a record passenger carriage of more than 26 million, a capacity of 79%, and record revenues, operating profits, and net profits of $17,775 million, $2,692 million, and $2,157 million, respectively. The Board of Directors also recommends a final dividend of 28 cents per share for the financial year.'

In [27]:
weather_qa_result  = agent.invoke("how is the weather in singapore today in celsius?")
weather_qa_result['output'] # Should get the answer from the Google-search Tool



> Entering new AgentExecutor chain...
 I should use the search engine to find the current weather in Singapore.
Action: Search
Action Input: "weather in Singapore in celsius"
Observation: {'type': 'weather_result', 'temperature': '28', 'unit': 'Celsius', 'precipitation': '4%', 'humidity': '80%', 'wind': '8 km/h', 'location': 'Singapore', 'date': 'Tuesday 1:00 AM', 'weather': 'Mostly cloudy'}
Thought: I should check the date and time of the weather report to ensure it is current.
Action: Search
Action Input: "weather in Singapore in celsius"
Observation: {'type': 'weather_result', 'temperature': '28', 'unit': 'Celsius', 'precipitation': '4%', 'humidity': '80%', 'wind': '8 km/h', 'location': 'Singapore', 'date': 'Tuesday 1:00 AM', 'weather': 'Mostly cloudy'}
Thought: I now know the final answer.
Final Answer: The current weather in Singapore is mostly cloudy with a temperature of 28 degrees Celsius.

> Finished chain.


'The current weather in Singapore is mostly cloudy with a temperature of 28 degrees Celsius.'